![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)

Para el procesamiento y operaciones de Big Data, la primer tarea es la recuperación de la información, entendida como la forma en que la información deseada por el usuario es especificada y recuperada del sistema del almacenamiento.

Esta operación se realiza a través de consultas, las cuales utilizan un lenguaje (*Query Language*) que permite especificar la información deseada; estos lenguajes son **declarativos**, es decir, el usuario indica qué necesita en vez de como obtenerlo. No es necesario escribir un programa que indique que archivo se va a abrir, qué bytes tener en cuenta, cuáles no, el tipo de codificación, entre otras.

Iniciaremos revisando la recuperación de datos relacionales, donde el lenguaje más utilizado es *Structured Query Language* o SQL con el cual se definen consultas de la forma: 
```sql
SELECT  atributos
FROM    tablas
WHERE   condiciones
```
Donde los atributos son los nombres de las columnas de la tabla.

En relación a lo visto en los modelos de datos, y las operaciones con conjuntos de datos, podríamos ver este tipo de consultas como una selección de los elementos de la tabla que cumplen cierta condición, y la posterior proyección de los atributos deseados.

En este notebook veremos las dos formas que plantea el framework de Apache Spark para la recuperación de información para datos relacionales: los DataFrames y SparkSQL.

# Relational data wrangling with Apache Spark

<small>Adapted from [GitHub](https://github.com/weberdavid/pyspark_course/)</small>

## Google Colaboratory environment set up

In [ ]:
# Download Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Next, we will install Apache Spark 3.0.1 with Hadoop 2.7 from here.
!wget https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# Now, we just need to unzip that folder.
!tar xf spark-3.3.2-bin-hadoop3.tgz

# Setting JVM and Spark path variables
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# Installing required packages
!pip install pyspark==3.3.2
!pip install findspark

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

import findspark
findspark.init()

from pyspark.sql import Window
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as fct

## Using Spark DataFrames

### Create Spark session and import the data

In [ ]:
ss = (SparkSession
      .builder
      .appName("wrangling_with_data")
      .getOrCreate())

path = "../Data/sparkify-log.json"
user_data = ss.read.json(path)

### Data Exploration

In [ ]:
# General information
user_data.printSchema()
(user_data
 .describe() # describe = variable + datatype; takes variable as parameter
 .show()) # show = count, mean, stddev, min, max for each var  

In [ ]:
# Counts number of users
user_data.count()

In [ ]:
# Selects variable/column and drops duplicates
(user_data
 .select("page")
 .dropDuplicates()
 .sort(fct.desc("page"))
 .show())

In [ ]:
# Select specific variables from a single user
(user_data
 .select(["userID", "firstname", "lastname", "level"])
 .where(user_data.userId == "1046").collect())

In [ ]:
# Calculating stuff with user defined function
get_hour = fct.udf(lambda x: dt.datetime.fromtimestamp(x/1000.0).hour)
# Create new column "hour" and fill with calculated hour of timestamp
user_data = user_data.withColumn("hour", get_hour(user_data.ts))
# Show first rows
user_data.head()

In [ ]:
# How many songs are listened per hour?
songs_in_hour = (user_data
                 .filter(user_data.page == "NextSong")
                 .groupby(user_data.hour)
                 .count()
                 .orderBy(user_data.hour.cast("float")))
songs_in_hour.show()

### Data Visualization

In [ ]:
# Convert to pandas df for visualization
songs_in_hour_pd = songs_in_hour.toPandas()
# Convert hour to numeric
songs_in_hour_pd.hour = pd.to_numeric(songs_in_hour_pd.hour)

In [ ]:
# Scatterplot
plt.scatter(songs_in_hour_pd["hour"], songs_in_hour_pd["count"])
plt.xlim(-1, 24);
plt.ylim(0, 1.2 * max(songs_in_hour_pd["count"]))
plt.xlabel("Hour")
plt.ylabel("Count of Songs Played")

### Data Operations

In [ ]:
# Look for and drop missing values: only NAs in userId and or sessionId
valid_users = user_data.dropna(how = "any", subset = ["userId", "sessionId"])
valid_users.count()

In [ ]:
# Drop duplicates: drop Dup, sort after User ID
(valid_users
 .select("userId")
 .dropDuplicates()
 .sort("userId")
 .show())

In [ ]:
# Drop empty strings
valid_users = valid_users.filter(valid_users["userId"] != "")
valid_users.count()

In [ ]:
# Are there users downgrading accounts?
valid_users.filter(valid_users["page"] == "Submit Downgrade").show()

# Give downgraders a flag; first create function; second give flag to all users
flag_downgrade_event = fct.udf(lambda x: 1 if x == "Submit Downgrade" else 0, IntegerType())
# withColumn = creates new column, or takes current and pastes values in
valid_users = valid_users.withColumn("downgraded", flag_downgrade_event("page"))

In [ ]:
# Work with Window
windowval = Window.partitionBy("userId").orderBy(fct.desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
# New column phase, that is the sum of downgraded
valid_users = valid_users.withColumn("phase", fct.sum("downgraded").over(windowval))
# Select variables of user, sort and collect data
(valid_users
 .select(["userId", "firstname", "ts", "page", "level", "phase"])
 .where(user_data.userId == "1138")
 .sort("ts")
 .collect())

## Using Spark SQL

In [ ]:
# Create a temporary view to run SQL queries
user_data.createOrReplaceTempView("user_data_table")

### Create queries

In [ ]:
ss.sql('''
       SELECT *
       FROM user_data_table
       LIMIT 2
       ''').show()  # .show is required to surpass lazyevaluation of spark

In [ ]:

ss.sql('''
       SELECT userId, count(page)
       FROM user_data_table
       GROUP BY userId
       ''').show()

In [ ]:
ss.sql('''
       SELECT userId, firstname, page, song
       FROM user_data_table
       WHERE userId = '1046'
       ''').collect()  # Attention - difference between show and collect

### Using user defined functions

In [ ]:
# Must be registered first
ss.udf.register("get_hour", lambda x: int(dt.datetime.fromtimestamp(x / 1000).hour))

In [ ]:
ss.sql('''
       SELECT userId, AVG(get_hour(ts)) as avg_hour
       FROM user_data_table
       GROUP BY userId
       ''').show()

## Exercises

#### Which page did user id "" NOT visit?

Double-click for one possible solution.

<!--
(user_data
 .where(user_data["userId"] == "")
 .groupby(user_data.page)
 .count()
 .show())

ss.sql('''
       SELECT page
       FROM user_data_table
       WHERE userId = ""
       GROUP BY page
       ''').show()
-->

### How many female users are in the dataset?

Double-click for one possible solution.
<!--
(user_data
 .select(["userId"])
 .dropDuplicates()
 .where(user_data["gender"] == "F")
 .count())

ss.sql('''
       SELECT count(distinct(userId))
       FROM user_data_table
       WHERE gender = 'F'
       ''').show()
-->

### From the most played artist, how many songs were played?
Double-click for one possible solution.
<!--
(user_data
 .filter(user_data.page == "NextSong")
 .select("Artist")
 .groupby("Artist")
 .count()
 .sort(fct.desc("count"))
 .show(1))

ss.sql('''
       SELECT artist, count(artist) as count
       FROM user_data_table
       WHERE page = 'NextSong'
       GROUP BY artist
       ORDER BY count DESC
       LIMIT 1
       ''').show()
-->